<a href="https://colab.research.google.com/github/marastika/-Environmental-Risk-Assessment-ERA-/blob/main/Semantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [9]:
import pandas as pd

# Use pd.read_excel to read Excel files
dataset = pd.read_excel('/content/contoh_data_with_sentiment.xlsx')

In [10]:
dataset.shape

(300, 8)

In [11]:
dataset.head()

,Nama url toko,Toko,Product ID,Product Name,Rating,Review Text,Reviewer Name,Sentiment
0,https://www.tokopedia.com/lpcomp,LPComp,1847675209,Heatsink SSD M.2 NVMe NGFF M2 Pendingin Alumin...,5,"packing aman, sesuai deskripsi",A***e,Neutral
1,https://www.tokopedia.com/lpcomp,LPComp,27279767,Keyboard Protector Netbook Bening 10 12 14 15....,5,Barang sesuai deskripsi dan berfungsi dengan b...,Eddy,Neutral
2,https://www.tokopedia.com/lpcomp,LPComp,15108854831,Mousepad Desk Mat Besar Jumbo XL 2,5,Barang sesuai deskripsi dan berfungsi dengan b...,Eddy,Neutral
3,https://www.tokopedia.com/lpcomp,LPComp,27279767,Keyboard Protector Netbook Bening 10 12 14 15....,5,"packing aman, ukuran sesuai, warna oke",elia,Neutral
4,https://www.tokopedia.com/lpcomp,LPComp,7054761965,Laptop Keyboard Blanket Microfiber Screen Clea...,5,"packing aman, material bagus, mantap",elia,Neutral


In [12]:
import spacy
import pandas as pd
nlp = spacy.load('en_core_web_sm')
from spacy import displacy


In [13]:
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
from spacy.lang.en.stop_words import STOP_WORDS

In [15]:
def text_data_cleaning(sentence):
    doc = nlp(sentence)

    tokens = []
    for token in doc:
        if token.lemma_ != '-PRON-':
            temp = token.lemma_.lower().strip()

        else:
            temp = token.lower_
        tokens.append(temp)

    cleaned_tokens= []
    for token in tokens:
        if token not in STOP_WORDS and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [18]:
# Replace this line:
# text_data_cleaning(dataset["Review"][i])
# with this line:
text_data_cleaning(dataset["Review Text"][i]) # Column name is likely "Review Text"

['pack', 'aman', 'sesuai', 'deskripsi']

In [21]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
corpus=[]

for i in range(0, 900):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

KeyError: 'Review'

In [23]:
# Replace this line:
# text_data_cleaning(dataset["Review"][i])
# with this line:
text_data_cleaning(dataset["Review Text"][i]) # Column name is likely "Review Text"

['pack', 'aman', 'sesuai', 'deskripsi']

In [26]:
corpus=[]

# Change the loop range to the actual number of rows in the dataset
for i in range(0, len(dataset)):
  # Change 'Review' to 'Review Text'
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review Text'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [27]:
corpus

['pack aman sesuai deskripsi',
 'barang sesuai deskripsi dan berfungsi dengan baik dipack dengan aman terima kasih gan',
 'barang sesuai deskripsi dan berfungsi dengan baik dipack dengan aman terima kasih gan',
 'pack aman ukuran sesuai warna oke',
 'pack aman materi bagu mantap',
 'pack aman motifnya keren banget ukurannya besar',
 'pack aman barang berfungsi dengan baik semoga awet',
 'bagu tebel kuylah bungku',
 'berfungsi dengan baik untuk sekrup macbook pengiriman aman dengan kardu kecil kokoh dikirim dengan cepat',
 'produk sesuai deskripsi sudah dite dan berfungsi dengan baik prose n pengiriman cepat pack aman n baik terimakasih',
 'pack mantaaap barang cepat sampai makasih',
 'secinta itu sama toko ini dan bnr rekomen bgt toko ini di kirim di hari yg sama sama pesanan hari ini pesen bsk samp luvv barangnya sesuai dengan deskripsi kualitasnya bagu dan pengirimannya cepat pelayanan ramah dan pack rapi barang sampai dalam kondisi sempurna harganya juga terjangkau dengan kualita ya

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1420)

In [29]:
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

In [30]:
# Saving BoW dictionary to later use in prediction
import pickle
bow_path = '/content/drive/MyDrive/data/c1_BoW_Sentiment_Model.pkl'
pickle.dump(cv, open(bow_path, "wb"))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/c1_BoW_Sentiment_Model.pkl'

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [32]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [33]:
# Exporting NB Classifier to later use in prediction
import joblib
joblib.dump(classifier, '/content/drive/MyDrive/data/c2_Classifier_Sentiment_Model')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/c2_Classifier_Sentiment_Model'

In [34]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

[[18 24]
 [ 5 13]]


0.5166666666666667

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(300, 654)

In [37]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

['Positive' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Positive' 'Neutral'
 'Positive' 'Positive' 'Neutral' 'Neutral' 'Positive' 'Positive'
 'Positive' 'Positive' 'Positive' 'Positive' 'Positive' 'Positive'
 'Positive' 'Positive' 'Positive' 'Positive' 'Positive' 'Positive'
 'Positive' 'Positive' 'Positive' 'Positive' 'Positive' 'Positive'
 'Neutral' 'Positive' 'Positive' 'Positive' 'Positive' 'Positive'
 'Positive' 'Positive' 'Neutral' 'Positive' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Positive' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Positive' 'Neutral'
 'Neutral' 'Neutral' 'Positive' 'Positive' 'Positive' 'Positive'
 'Positive' 'Neutral' 'Positive' 'Positive' 'Positive' 'Positive'
 'Positive' 'Positive' 'Positive' 'Positive' 'Neutral' 'Positive'
 'Neutral' 'Neutral' 'Neutral' 'Positive' 'Neutral' 'Positive' 'Positive'
 'Neutral' 'Neutral' 'Neutral' 'Positive' 'Neutral' 'Positive' 'Positive'
 'Neutral' 'Positive' 'Positive'

In [38]:
dataset['predicted_label'] = y_pred.tolist()
dataset.head()

,Nama url toko,Toko,Product ID,Product Name,Rating,Review Text,Reviewer Name,Sentiment,predicted_label
0,https://www.tokopedia.com/lpcomp,LPComp,1847675209,Heatsink SSD M.2 NVMe NGFF M2 Pendingin Alumin...,5,"packing aman, sesuai deskripsi",A***e,Neutral,Positive
1,https://www.tokopedia.com/lpcomp,LPComp,27279767,Keyboard Protector Netbook Bening 10 12 14 15....,5,Barang sesuai deskripsi dan berfungsi dengan b...,Eddy,Neutral,Neutral
2,https://www.tokopedia.com/lpcomp,LPComp,15108854831,Mousepad Desk Mat Besar Jumbo XL 2,5,Barang sesuai deskripsi dan berfungsi dengan b...,Eddy,Neutral,Neutral
3,https://www.tokopedia.com/lpcomp,LPComp,27279767,Keyboard Protector Netbook Bening 10 12 14 15....,5,"packing aman, ukuran sesuai, warna oke",elia,Neutral,Neutral
4,https://www.tokopedia.com/lpcomp,LPComp,7054761965,Laptop Keyboard Blanket Microfiber Screen Clea...,5,"packing aman, material bagus, mantap",elia,Neutral,Neutral


In [39]:
import pandas as pd
import numpy as np